---

- [Dataclasses](https://pydantic-docs.helpmanual.io/usage/devtools/)
- [generate_models_person_model]()
- [index_main and index_error]()

This next script can be found at the top of the [models.ipynb](models.ipynb)

In [5]:
# %load ../docs/examples/generate_models_person_model.py
# generated by datamodel-codegen:
#   filename:  person.json
#   timestamp: 2020-05-19T15:07:31+00:00
from __future__ import annotations
from typing import Any, List, Optional
from pydantic import BaseModel, Field, conint


class Pet(BaseModel):
    name: Optional[str] = None
    age: Optional[int] = None


class Person(BaseModel):
    first_name: str = Field(..., description="The person's first name.")
    last_name: str = Field(..., description="The person's last name.")
    age: Optional[conint(ge=0)] = Field(None, description='Age in years.')
    pets: Optional[List[Pet]] = None
    comment: Optional[Any] = None


---

Example of changing parameter values.

In [7]:
# %load ../docs/examples/hypothesis_property_based_test.py
import typing
from hypothesis import given, strategies as st
from pydantic import BaseModel, EmailStr, PaymentCardNumber, PositiveFloat


class Model(BaseModel):
    card: PaymentCardNumber
    price: PositiveFloat
    users: typing.List[EmailStr]


@given(st.builds(Model))
def test_property(instance):
    # Hypothesis calls this test function many times with varied Models,
    # so you can write a test that should pass given *any* instance.
    assert 0 < instance.price
    assert all('@' in email for email in instance.users)


@given(st.builds(Model, price=st.floats(100, 200)))
def test_with_discount(instance):
    # This test shows how you can override specific fields,
    # and let Hypothesis fill in any you don't care about.
    assert 100 <= instance.price <= 200


---

Simple model and it's validator

In [13]:
# %load ../docs/examples/index_main.py
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name = 'John Doe'
    signup_ts: Optional[datetime] = None
    friends: List[int] = []


external_data = {
    'id': '123',
    'signup_ts': '2019-06-01 12:22',
    'friends': [1, 2, '3'],
}
user = User(**external_data)
print(user.id)
print(repr(user.signup_ts))
print(user.friends)
print(user.dict())


123
datetime.datetime(2019, 6, 1, 12, 22)
[1, 2, 3]
{'id': 123, 'signup_ts': datetime.datetime(2019, 6, 1, 12, 22), 'friends': [1, 2, 3], 'name': 'John Doe'}


In [14]:
# %load ../docs/examples/index_error.py
# output-json
#from index_main import User  # see above

# ignore-above
from pydantic import ValidationError

try:
    User(signup_ts='broken', friends=[1, 2, 'not number'])
except ValidationError as e:
    print(e.json())


[
  {
    "loc": [
      "id"
    ],
    "msg": "field required",
    "type": "value_error.missing"
  },
  {
    "loc": [
      "signup_ts"
    ],
    "msg": "invalid datetime format",
    "type": "value_error.datetime"
  },
  {
    "loc": [
      "friends",
      2
    ],
    "msg": "value is not a valid integer",
    "type": "type_error.integer"
  }
]


---

print a field that is not a part of the model

In [2]:
# %load ../docs/examples/mypy_main.py
# dont-execute
from datetime import datetime
from typing import List, Optional
from pydantic import BaseModel, NoneStr


class Model(BaseModel):
    age: int
    first_name = 'John'
    last_name: NoneStr = None
    signup_ts: Optional[datetime] = None
    list_of_ints: List[int]


m = Model(age=42, list_of_ints=[1, '2', b'3'])
print(m.middle_name)  # not a model field!
Model()  # will raise a validation error for age and list_of_ints


AttributeError: 'Model' object has no attribute 'middle_name'

---

parse_obj_as JSON -> Class instance

In [4]:
# %load ../docs/examples/parse_obj_as.py
from typing import List

from pydantic import BaseModel, parse_obj_as


class Item(BaseModel):
    id: int
    name: str


# `item_data` could come from an API call, eg., via something like:
# item_data = requests.get('https://my-api.com/items').json()
item_data = [{'id': 1, 'name': 'My Item'}]

items = parse_obj_as(List[Item], item_data)
print(items)


[Item(id=1, name='My Item')]


---

